In [ ]:
# Unified Intent Detection + Entity Extraction using FLAN-T5

!pip install -q transformers datasets accelerate

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import pandas as pd
import torch

# 1. Sample Data
examples = [
    {"input": "Send 500 to Arun", "target": '{"intent": "money_transfer", "recipient": "Arun", "amount": 500}'},
    {"input": "Book a cab to airport", "target": '{"intent": "book_ride", "destination": "airport"}'},
    {"input": "Recharge my phone with 199", "target": '{"intent": "recharge", "amount": 199}'},
    {"input": "Pay electricity bill", "target": '{"intent": "bill_payment", "type": "electricity"}'},
    {"input": "Transfer 1000 to Neha", "target": '{"intent": "money_transfer", "recipient": "Neha", "amount": 1000}'}
]

# Convert to HF dataset
df = pd.DataFrame(examples)
dataset = Dataset.from_pandas(df)

# 2. Tokenizer and Model
model_checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# 3. Tokenization Function
def preprocess(example):
    model_inputs = tokenizer(example['input'], max_length=64, truncation=True)
    labels = tokenizer(example['target'], max_length=64, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = dataset.map(preprocess)

# 4. Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_intent_model",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=1,
    evaluation_strategy="no",
    save_strategy="no",
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 5. Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# 6. Train
trainer.train()

# 7. Inference
def predict(text):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 🔍 Try it
print(predict("Pay 200 to mom"))
print(predict("Recharge Jio with 299"))